#### Runable 协议

In [12]:
from langchain_ollama import OllamaLLM
# from langchain_openai import ChatOpenAI

model = OllamaLLM(model="deepseek-r1:1.5b", base_url="http://localhost:11434")

chunks = []

for chunk in model.stream("天空是什么颜色"):
    chunks.append(chunk)
    print(chunk, end="|", flush=True)
    # print(chunk.content, end="|", flush=True)
    
print("\n",dir(chunk)) # chunk 是一个字符串类型，并不像OpenAI的ChatOpenAI那样有content属性

<think>|

|</think>|

|天空|的颜色|因|人|而|异|，|既|然是|“|空|”的|话|，|自然|不会|有任何|色彩|。|然而|，在|不同的|文化|中|，|我们|可能会|赋予|天空|独特的|“|颜色|”|以|象征|意义|。

|-| **|东方|文化|**|：|在|东方|文化|中|，|天空|是|柔和|的|橙|红色|、|黄色|或|橘|色|，|象征|着|希望|和|繁荣|。
|-| **|西方|文化|**|：|在|西方|文化|中|，|天空|通常|被认为|是最|亮|的|，|有时|呈现出|蓝色|、|紫色|或|金色|，|象征|着|光明|和|高|洁|。

|每个|地方|都有|独特的|文化和|自然|选择|赋予|天空|不同的|“|颜色|”，|但|这些|颜色|本质上|都是|空|的|。||
 ['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islow

In [16]:
chunks[5],chunks[10],chunks[15]

('的颜色', '，', '”的')

##### 异步调用

In [ ]:
from langchain_ollama import OllamaLLM
import asyncio
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import HumanMessagePromptTemplate

# ChatPromptTemplate.from_messages()期望传入的格式极为严格，
# 每个message必须包含role和content属性，并且content属性必须是一个字符串。
# 如果传入的message不满足这些要求，ChatPromptTemplate.from_messages()会抛出一个ValueError异常。


# 修改这部分，使用HumanMessagePromptTemplate创建符合格式的消息
prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("给我讲一个关于{topic}的故事")
])

model = OllamaLLM(model="deepseek-r1:1.5b", base_url="http://localhost:11434")
# parser = StrOutputParser()
# chain = prompt | model | parser
chain = prompt | model

async def async_stream():
    async for chunk in chain.stream({"topic":"天空"}):
        print(chunk, end="|", flush=True)

# asyncio.run(async_stream()) # chain.stream({"topic":"天空"}) 返回的是一个普通的生成器对象，
# 而不是一个实现了异步迭代协议（即具有 __aiter__ 和 __anext__ 方法）的异步可迭代对象

# 因此，在调用 asyncio.run() 时，Python 会尝试将生成器对象转换为异步可迭代对象，
# 但由于生成器对象本身并不实现异步迭代协议，因此会引发 TypeError 异常。

In [30]:
import nest_asyncio
nest_asyncio.apply()
from langchain_ollama import OllamaLLM
import asyncio
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import HumanMessagePromptTemplate


# 包装生成器为异步可迭代对象的类
class AsyncGeneratorWrapper:
    def __init__(self, gen):
        self.gen = gen

    def __aiter__(self):
        return self

    async def __anext__(self):
        try:
            return next(self.gen)
        except StopIteration:
            raise StopAsyncIteration


prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("给我讲一个关于{topic}的故事")
])

model = OllamaLLM(model="deepseek-r1:1.5b", base_url="http://localhost:11434")
parser = StrOutputParser()
chain = prompt | model | parser


async def async_stream():
    gen = chain.stream({"topic": "天空"})
    async_gen = AsyncGeneratorWrapper(gen)
    async for chunk in async_gen:
        print(chunk, end="|", flush=True)


asyncio.run(async_stream())

<think>|
|嗯|，|用户|让我|讲|一个|关于|天空|的故事|。|首先|，|我|得|想想|这个|故事|应该|是什么|样的|。|可能|从|自然|景象|开始|，|然后|引|出|一些|象征|意义|或者|寓意|。|天空|通常|象征|着|自由|、|广阔|和|神秘|，|所以|故事|里|可以|突出|这些|元素|。

|用户|的需求|是|讲|一个|简单|的故事|，|所以我|需要|选择|一个|不|复杂|但|又有|深度|的主题|。|或许|可以从|日常生活|中的|某个|瞬间|切入|，|比如|下雨|天|，|这样|容易|引起|共鸣|。|同时|，|加入|一些|细节|会让|故事|更|生动|。

|接下来|，|考虑|主角|的选择|。|可能|是一个|普通人|，|比如|一个|喜欢|自然|的朋友|，|或者|有|特殊|能力|的人|。|假设|是|朋友|的话|，|可以|描述|她|因为|下雨|而|发现|的|天空|之|谜|，|这样|既|真实|又|容易|展开|情节|。

|然后|，|结构|上|应该|有一个|引|人|入|胜|的|开头|，|中间|的情|节|发展|，|最后|有一个|感|人的|结尾|。|引|言|部分|可以通过|一个小|插|曲|引入|，|比如|她|从|不|关|窗户|开始|，|这样|自然|地|引|出|她的|探索|之旅|。

|在|描写|场景|时|，|要|细致|刻画|环境|，|比如|雨|后的|天空|、|树叶|上的|露|珠|、|泥土|的颜色|等|细节|，|让|读者|有|身|临|其|境|的感觉|。|同时|，|加入|一些|象征|性的|元素|，|比如|露|珠|代表|希望|，|沙|粒|象征|寂静|和|黑暗|中的|光明|，|这样|故事|更有|深度|。

|最后|，|结尾|部分|需要|给人|留下|深刻|的印象|，|可能|是一个|温暖|的|感悟|或者|令人|反思|的主题|。|比如|她|发现|天空|的秘密|，|学会了|在|困难|时|保持|乐观|，|这样的|结局|既|感人|又|有力|。

|总的来说|，|我|需要|构建|一个|简单|但|富有|层次|感|的故事|，|通过|自然|元素|和|隐|喻|，|传达|出|关于|天空|和|内心的|和谐|。|这样|不仅|满足|了|用户|的需求|，|还能|给|读者|带来|启发|和|情感|上的|共鸣|。
|</think>|

|##| |《|雨|后的|秘密|》|

|我|从|不|关|窗户|。

